# 📊 Data Exploration

Load OHLCV data from `HistoricalDataStore`, compute features via `FeatureStore`, and visualize.

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import matplotlib.pyplot as plt

from backend.etl.data_access.storage.HistoricalDataStore import HistoricalDataStore
from backend.scitus.features import FeatureStore

## 1. Load Raw Data

In [ ]:
store = HistoricalDataStore()

ASSET = "BTCUSDT"
START = "2024-01-01"
END   = "2024-12-31"

df = store.query(asset_id=ASSET, start_date=START, end_date=END)
print(f"Loaded {len(df)} rows for {ASSET}")
df.head()

## 2. Compute Features

In [ ]:
fs = FeatureStore(data_store=store)

features_df = fs.compute(
    asset_id=ASSET,
    feature_names=["rsi_14", "sma_20", "sma_50"],
    start=START,
    end=END,
)
features_df.head(10)

## 3. Plot Candlesticks + SMA Overlay

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True,
                                gridspec_kw={'height_ratios': [3, 1]})

# Price + SMAs
ax1.plot(df['timestamp'], df['close'], label='Close', linewidth=0.8)

if not features_df.empty:
    ax1.plot(df['timestamp'].iloc[-len(features_df):], features_df['sma_20'], label='SMA 20', alpha=0.7)
    ax1.plot(df['timestamp'].iloc[-len(features_df):], features_df['sma_50'], label='SMA 50', alpha=0.7)

ax1.set_title(f'{ASSET} Price & Moving Averages')
ax1.legend()
ax1.grid(True, alpha=0.3)

# RSI
if not features_df.empty and 'rsi_14' in features_df.columns:
    ax2.plot(df['timestamp'].iloc[-len(features_df):], features_df['rsi_14'], color='purple', linewidth=0.8)
    ax2.axhline(70, color='red', linestyle='--', alpha=0.5)
    ax2.axhline(30, color='green', linestyle='--', alpha=0.5)

ax2.set_title('RSI (14)')
ax2.set_ylim(0, 100)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()